# Data Cleaning and Handling Missing Values

1. Total Missing Values Counts
2. Handling Missing Data
3. Comparing Missing Data by Year Range
4. Data Visualization by Year Range
5. Key Considerations for Selecting the Optimal Year Range

#### Load the data

In [2]:
# Load the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Load your dataset
df = pd.read_csv('../dsi_team_22/data/raw/pharma_spending_data.csv')

## 1. Total Missing Values per Country (1970 to 2016)
| country code | country name    | # of observations | # of missing values | % missing  |
|:------------:|:---------------:|:-----------------:|:-------------------:|-----------:|
| RUS          | Russia          | 1                 | 46                  | 97.87%     |
| ISR          | Israel          | 8                 | 39                  | 82.98%     |
| TUR          | Turkey          | 11                | 36                  | 76.60%     |
| LTU          | Lithuania       | 12                | 35                  | 74.47%     |
| LVA          | Latvia          | 12                | 35                  | 74.47%     |
| SVN          | Slovenia        | 14                | 33                  | 70.21%     |
| POL          | Poland          | 14                | 33                  | 70.21%     |
| USA          | United States   | 16                | 31                  | 65.96%     |
| EST          | Estonia         | 17                | 30                  | 63.83%     |
| SVK          | Slovakia        | 17                | 30                  | 63.83%     |
| MEX          | Mexico          | 17                | 30                  | 63.83%     |
| LUX          | Luxembourg      | 21                | 26                  | 55.32%     |
| HUN          | Hungary         | 22                | 25                  | 53.19%     |
| NZL          | New Zealand     | 24                | 23                  | 48.94%     |
| CZE          | Czech Republic  | 26                | 21                  | 44.68%     |
| AUT          | Austria         | 26                | 21                  | 44.68%     |
| GRC          | Greece          | 27                | 20                  | 42.55%     |
| ITA          | Italy           | 29                | 18                  | 38.30%     |
| FRA          | France          | 30                | 17                  | 36.17%     |
| CHE          | Switzerland     | 31                | 16                  | 34.04%     |
| GBR          | United Kingdom  | 31                | 16                  | 34.04%     |
| JPN          | Japan           | 32                | 15                  | 31.91%     |
| ESP          | Spain           | 32                | 15                  | 31.91%     |
| DNK          | Denmark         | 36                | 11                  | 23.40%     |
| PRT          | Portugal        | 39                | 8                   | 17.02%     |
| IRL          | Ireland         | 41                | 6                   | 12.77%     |
| BEL          | Belgium         | 41                | 6                   | 12.77%     |
| NLD          | Netherlands     | 43                | 4                   | 8.51%      |
| AUS          | Australia       | 44                | 3                   | 6.38%      |
| DEU          | Germany         | 45                | 2                   | 4.26%      |
| CAN          | Canada          | 45                | 2                   | 4.26%      |
| SWE          | Sweden          | 46                | 1                   | 2.13%      |
| ISL          | Iceland         | 46                | 1                   | 2.13%      |
| FIN          | Finland         | 46                | 1                   | 2.13%      |
| NOR          | Norway          | 47                | 0                   |  0%        |
| KOR          | South Korea     | 47                | 0                   |  0%        |


#### Supporting Python Code:

In [ ]:
# Define the range of years and get unique country codes
years = np.arange(1970, 2017)
countries = df['LOCATION'].unique()

# Create a reference DataFrame with all combinations of countries and years
reference_df = pd.MultiIndex.from_product([countries, years], names=['LOCATION', 'TIME']).to_frame(index=False)

# Merge the reference DataFrame with the original DataFrame to find missing combinations
merged_df = reference_df.merge(df, on=['LOCATION', 'TIME'], how='left', indicator=True)

# Identify rows where the merge indicator shows that the row is missing in the original DataFrame
missing_rows = merged_df[merged_df['_merge'] == 'left_only']

# Display the missing country-year combinations
missing_combinations = missing_rows[['LOCATION', 'TIME']]
print("Missing country-year combinations:")
print(missing_combinations)

# Save the file
missing_combinations.to_csv('../../dsi_team_22/data/processed/missing_values.csv', index=False)

In [13]:
# Load the missive_values.csv dataset
df_missing = pd.read_csv('../../dsi_team_22/data/processed/missing_values.csv')

In [ ]:
# Total number of possible observations per country
total_possible_observations = 47

# Count the number of missing years for each country
missing_years_count = df_missing['LOCATION'].value_counts()

# Calculate the percentage of missing values for each country
missing_percentage = (missing_years_count / total_possible_observations) * 100

# Display the results
for country, count in missing_years_count.items():
    percentage = (count / total_possible_observations) * 100
    print(f"{country}: {count} missing years ({percentage:.2f}% missing)")


## 2. Handling Missing Values


### Understanding the Problems with Incomplete Data

When we analyze data to make decisions, having complete information is crucial as there are negative consequences when dealing with incomplete data sets. Out of the 36 countries covered in our data set, only 2 countries have complete data from the total year range from 1970 to 2016. The remaining 34 countries have missing data for some years. Understanding these issues is the first step in addressing them.

#### Here are some problems this can cause:

| Problem | What It Means | Impact |
|:-------:|-------|------|
| Biased Results | Imagine trying to understand everyone's opinion in a meeting, but only hearing from a few people. Your conclusions might not represent the whole group accurately. | If we analyze only the available data, our insights might be skewed. We could overestimate or underestimate spending trends because the missing data could be different from the available data. |
| Reduced Accuracy | Think of it like a puzzle with missing pieces. Without all pieces, the picture is incomplete and unclear. | Incomplete data can lead to incorrect conclusions. For instance, we might miss important trends in pharmaceutical spending or fail to notice changes in spending patterns over time. |
| Limited Comparisons | It's like comparing students' performances when some report cards are missing grades for several subjects. | It becomes challenging to compare countries accurately. We may not be able to tell if one country is truly spending more on pharmaceuticals than another, or if it just appears that way because we lack complete data. |
| Misleading Trends | Imagine tracking your daily exercise but missing several days. You might wrongly conclude you exercised less frequently than you actually did. | Trends over time might be misleading. Missing data points can create false impressions of increases or decreases in pharmaceutical spending, making it hard to identify genuine patterns. |
| Decision-Making Challenges | It's like making a business decision with only partial financial reports. | Policy makers and stakeholders rely on accurate data to make informed decisions. Incomplete data can lead to suboptimal or incorrect decisions about health policies and funding. |
| Statistical Limitations | In statistics, more data generally leads to more reliable results, similar to having more survey responses giving a clearer picture. | With missing data, the statistical power of our analyses is weakened. This means we might not detect significant trends or differences that would be apparent with complete data. |


## 3. Comparing Complete Data by Year Range

### OVERVIEW:
The table below compares the completeness of data over different time ranges: 1970-2016, 2005-2015, 2009-2013 and 2009-2012. This comparison highlights the number of years, the number of countries with complete data, the total possible observations, the actual number of observations, and the missing data for each range.

TABLE #1
| Start Date | End Date | # of Years | Possible Observations | # of Observations | # Missing |
|:----:|:----:|:---:|:----:|:-----:|:------:|
| 1970 | 2016 | 47 | 1692 | 1036 | 656 |
| 2005 | 2015 | 11 | 396 | 352 | 44 |
| 2005 | 2014 | 10 | 360 | 322 | 38 |
| 2009 | 2013 | 5 | 180 | 161 | 19 |
| 2009 | 2012 | 4 | 144 | 128 | 16 |

The table below compares the number of eliminated countries based on the different time ranges explored in the table above. 

TABLE #2: 
| Start Date | End Date | # of Years | # Dropped Countries | # Missing after drop | List of Dropped Countries |
|:----:|:----:|:---:|:---:|:---:|:---|
| 2005 | 2015 | 11 | 1 | 33 | Turkey |
| 2005 | 2014 | 10 | 2 | 18 | Turkey, Russia |
| 2009 | 2013 | 5 | 3 | 4 | Turkey, Russia, New Zealand |
| 2009 | 2012 | 4 | 4 | 0 | Turkey, Russia, New Zealand, United Kingdom |

#### Supporting Python Code:

#### TABLE #1

In [21]:
def generate_observations_summary(df, start_date, end_date, total_countries):
    # Calculate the number of years in the time range
    number_of_years = end_date - start_date + 1

    # Calculate the possible number of observations per time range
    possible_observations = total_countries * number_of_years

    # Filter the DataFrame to include only the rows within the specified date range
    df_filtered = df[(df['TIME'] >= start_date) & (df['TIME'] <= end_date)]

    # Calculate the number of true observations
    number_of_observations = len(df_filtered)

    # Calculate the number of missing observations
    number_of_missing = possible_observations - number_of_observations

    # Create the Markdown table
    markdown_table = f"""
| {start_date} | {end_date} | {number_of_years} | {possible_observations} | {number_of_observations} | {number_of_missing} |
    """

    return markdown_table

In [ ]:
total_countries = 36

# TOTAL YEAR RANGE
start_date = 1970
end_date = 2016
# Generate the summary table
summary_table = generate_observations_summary(df, start_date, end_date, total_countries)

# YEAR RANGE #1
start_date_1 = 2005
end_date_1 = 2015
# Generate the summary table
summary_table_1 = generate_observations_summary(df, start_date_1, end_date_1, total_countries)

# YEAR RANGE #2
start_date_2 = 2005
end_date_2 = 2014
# Generate the summary table
summary_table_2 = generate_observations_summary(df, start_date_2, end_date_2, total_countries)

# YEAR RANGE #3
start_date_3 = 2009
end_date_3 = 2013
# Generate the summary table
summary_table_3 = generate_observations_summary(df, start_date_3, end_date_3, total_countries)

# YEAR RANGE #4
start_date_4 = 2009
end_date_4 = 2012
# Generate the summary table
summary_table_4 = generate_observations_summary(df, start_date_4, end_date_4, total_countries)

print("| Start Date | End Date | # of Years | Possible Observations | # of Observations | # Missing |")
print(summary_table,summary_table_1,summary_table_2,summary_table_3,summary_table_4)

#### TABLE #2:

In [18]:
def generate_dropped_summary(df, start_date, end_date, total_countries, num_dropped_countries):
    # Calculate the number of years in the time range
    number_of_years = end_date - start_date + 1

    # Calculate the possible number of observations per time range
    possible_observations = total_countries * number_of_years

    # Filter the DataFrame to include only the rows within the specified date range
    df_filtered = df[(df['TIME'] >= start_date) & (df['TIME'] <= end_date)]

    # Calculate the number of true observations
    number_of_observations = len(df_filtered)

    # Calculate the number of missing observations
    number_of_missing = possible_observations - number_of_observations

    # Calculate the number of missing observations after dropping certain countries
    num_missing_dropped = number_of_missing - (number_of_years * num_dropped_countries)

    # Create the Markdown table
    markdown_table = f"""
| {start_date} | {end_date} | {number_of_years} | {num_dropped_countries} | {num_missing_dropped} |
    """

    return markdown_table

In [ ]:
total_countries = 36

# YEAR RANGE #1
start_date_1 = 2005
end_date_1 = 2015
num_dropped_countries_1 = 1

# YEAR RANGE #2
start_date_2 = 2005
end_date_2 = 2014
num_dropped_countries_2 = 2

# YEAR RANGE #3
start_date_3 = 2009
end_date_3 = 2013
num_dropped_countries_3 = 3

# YEAR RANGE #4
start_date_4 = 2009
end_date_4 = 2012
num_dropped_countries_4 = 4

# Generate the summary tables
summary_table_drop_1 = generate_dropped_summary(df, start_date_1, end_date_1, total_countries, num_dropped_countries_1)
summary_table_drop_2 = generate_dropped_summary(df, start_date_2, end_date_2, total_countries, num_dropped_countries_2)
summary_table_drop_3 = generate_dropped_summary(df, start_date_3, end_date_3, total_countries, num_dropped_countries_3)
summary_table_drop_4 = generate_dropped_summary(df, start_date_4, end_date_4, total_countries, num_dropped_countries_4)
print('| Start Date | End Date | # of Years | # Dropped Countries | # Missing after drop |')
print(summary_table_drop_1,summary_table_drop_2,summary_table_drop_3,summary_table_drop_4)

## 3. Comparing Missing Data by Year Range

#### TIME RANGE #1: Total Missing Values per Country (2005 to 2015)
| Country Code | Country Name | # of Observations | # of Missing Values | % Missing  |
|:------------:|:------------:|:-----------------:|:-------------------:|-----------:|
| TUR          | Turkey       | 0                 | 11                  | 100.00%    |
| RUS          | Russia       | 1                 | 10                  | 90.91%     |
| NZL          | New Zealand  | 3                 | 8                   | 72.73%     |
| GBR          | United Kingdom | 3                 | 8                   | 72.73%     |
| ISR          | Israel       | 8                 | 3                   | 27.27%     |
| AUS          | Australia    | 10                | 1                   | 9.09%      |
| CAN          | Canada       | 10                | 1                   | 9.09%      |
| GRC          | Greece       | 10                | 1                   | 9.09%      |
| JPN          | Japan        | 10                | 1                   | 9.09%      |

#### TIME RANGE #2: Total Missing Values per Country (2005 to 2014)
| Country Code | Country Name     | # of Observations | # of Missing Values | % Missing  |
|:------------:|:----------------:|:-----------------:|:-------------------:|-----------:|
| TUR          | Turkey           | 0                 | 10                  | 100.00%    |
| RUS          | Russia           | 0                 | 10                  | 100.00%    |
| GBR          | United Kingdom   | 2                 | 8                   | 80.00%     |
| NZL          | New Zealand      | 3                 | 7                   | 70.00%     |
| ISR          | Israel           | 8                 | 2                   | 20.00%     |
| GRC          | Greece           | 9                 | 1                   | 10.00%     |

#### TIME RANGE #3: Total Missing Values per Country (2009 to 2013)
| Country Code | Country Name     | # of Observations | # of Missing Values | % Missing  |
|:------------:|:----------------:|:-----------------:|:-------------------:|-----------:|
| NZL          | New Zealand      | 0                 | 5                   | 100.00%    |
| TUR          | Turkey           | 0                 | 5                   | 100.00%    |
| RUS          | Russia           | 0                 | 5                   | 100.00%    |
| GBR          | United Kingdom   | 1                 | 4                   | 80.00%     |

#### TIME RANGE #4: Total Missing Values per Country (2009 to 2012)
| Country Code | Country Name     | # of Observations | # of Missing Values | % Missing  |
|:------------:|:----------------:|:-----------------:|:-------------------:|-----------:|
| NZL          | New Zealand      | 0                 | 5                   | 100.00%    |
| TUR          | Turkey           | 0                 | 5                   | 100.00%    |
| RUS          | Russia           | 0                 | 5                   | 100.00%    |
| GBR          | United Kingdom   | 0                 | 5                   | 100.00%     |

#### Supporting Python Code:

In [ ]:
# TIME RANGE #1: Exploring the 2005 - 2015

# Define the range of years we are interested in
start_year = 2005
end_year = 2015
filtered_years_count = end_year - start_year + 1  # Total number of years in the range

# Filter the dataframe to include only the years between 2005 and 2015
df_filtered = df_missing[(df_missing['TIME'] >= start_year) & (df_missing['TIME'] <= end_year)]

# Count the number of missing years for each country within the specified range
missing_years_count = df_filtered['LOCATION'].value_counts()

# Calculate the percentage of missing values for each country
total_possible_observations = filtered_years_count  # Total possible observations per country in the filtered range

# Display the results
for country, count in missing_years_count.items():
    percentage = (count / total_possible_observations) * 100
    print(f"{country}: {count} missing years ({percentage:.2f}% missing)")

In [ ]:
# TIME RANGE #2: Exploring the 2005 - 2014

# Define the range of years we are interested in
start_year = 2005
end_year = 2014
filtered_years_count = end_year - start_year + 1  # Total number of years in the range

# Filter the dataframe to include only the years between 2005 and 2014
df_filtered = df_missing[(df_missing['TIME'] >= start_year) & (df_missing['TIME'] <= end_year)]

# Count the number of missing years for each country within the specified range
missing_years_count = df_filtered['LOCATION'].value_counts()

# Calculate the percentage of missing values for each country
total_possible_observations = filtered_years_count  # Total possible observations per country in the filtered range

# Display the results
for country, count in missing_years_count.items():
    percentage = (count / total_possible_observations) * 100
    print(f"{country}: {count} missing years ({percentage:.2f}% missing)")

In [ ]:
# TIME RANGE #3: Exploring the 2009 - 2013

# Define the range of years we are interested in
start_year = 2009
end_year = 2013
filtered_years_count = end_year - start_year + 1  # Total number of years in the range

# Filter the dataframe to include only the years between 2009 and 2013
df_filtered = df_missing[(df_missing['TIME'] >= start_year) & (df_missing['TIME'] <= end_year)]

# Count the number of missing years for each country within the specified range
missing_years_count = df_filtered['LOCATION'].value_counts()

# Calculate the percentage of missing values for each country
total_possible_observations = filtered_years_count  # Total possible observations per country in the filtered range

# Display the results
for country, count in missing_years_count.items():
    percentage = (count / total_possible_observations) * 100
    print(f"{country}: {count} missing years ({percentage:.2f}% missing)")

## 5. Data Visualization: Year Range Comparisons

![year_range_1.png](images/year_range_1.png)

![year_range_2.png](images/year_range_2.png)

![year_range_3.png](images/year_range_3.png)

#### Supporting Python Code:

In [ ]:
# Filter the DataFrame to include only the years 2005 to 2015
df_filtered = df[(df['TIME'] >= 2005) & (df['TIME'] <= 2015)]

# Create a pivot table with countries as rows and years as columns
pivot_table = df_filtered.pivot_table(index='LOCATION', columns='TIME', aggfunc='size', fill_value=0)

# Convert the pivot table to a binary format (presence or absence of observation)
pivot_table = pivot_table.applymap(lambda x: 1 if x > 0 else 0)

# Count the number of observations per country
observation_counts = pivot_table.sum(axis=1)

# Sort the countries by the number of observations
sorted_countries = observation_counts.sort_values(ascending=False).index

# Reorder the pivot table based on the sorted countries
pivot_table_sorted = pivot_table.loc[sorted_countries]

# Plot the heatmap
plt.figure(figsize=(8, 8))
heatmap = sns.heatmap(pivot_table_sorted, cmap="YlGnBu", cbar=False, linewidths=.5)

# Add labels and title
heatmap.set_xlabel('Year', fontsize=12)
heatmap.set_ylabel('Country', fontsize=12)
heatmap.set_title('Observations of Each Country by Year (2005 - 2015)', fontsize=14)

# Display the heatmap
plt.show()

In [ ]:
# Filter the DataFrame to include only the years 2005 to 2014
df_filtered = df[(df['TIME'] >= 2005) & (df['TIME'] <= 2014)]

# Create a pivot table with countries as rows and years as columns
pivot_table = df_filtered.pivot_table(index='LOCATION', columns='TIME', aggfunc='size', fill_value=0)

# Convert the pivot table to a binary format (presence or absence of observation)
pivot_table = pivot_table.applymap(lambda x: 1 if x > 0 else 0)

# Count the number of observations per country
observation_counts = pivot_table.sum(axis=1)

# Sort the countries by the number of observations
sorted_countries = observation_counts.sort_values(ascending=False).index

# Reorder the pivot table based on the sorted countries
pivot_table_sorted = pivot_table.loc[sorted_countries]

# Plot the heatmap
plt.figure(figsize=(8, 8))
heatmap = sns.heatmap(pivot_table_sorted, cmap="YlGnBu", cbar=False, linewidths=.5)

# Add labels and title
heatmap.set_xlabel('Year', fontsize=12)
heatmap.set_ylabel('Country', fontsize=12)
heatmap.set_title('Observations of Each Country by Year (2005 - 2014)', fontsize=14)

# Display the heatmap
plt.show()

In [ ]:
# Filter the DataFrame to include only the years 2009 to 2013
df_filtered = df[(df['TIME'] >= 2009) & (df['TIME'] <= 2013)]

# Create a pivot table with countries as rows and years as columns
pivot_table = df_filtered.pivot_table(index='LOCATION', columns='TIME', aggfunc='size', fill_value=0)

# Convert the pivot table to a binary format (presence or absence of observation)
pivot_table = pivot_table.applymap(lambda x: 1 if x > 0 else 0)

# Count the number of observations per country
observation_counts = pivot_table.sum(axis=1)

# Sort the countries by the number of observations
sorted_countries = observation_counts.sort_values(ascending=False).index

# Reorder the pivot table based on the sorted countries
pivot_table_sorted = pivot_table.loc[sorted_countries]

# Plot the heatmap
plt.figure(figsize=(4, 8))
heatmap = sns.heatmap(pivot_table_sorted, cmap="YlGnBu", cbar=False, linewidths=.5)

# Add labels and title
heatmap.set_xlabel('Year', fontsize=12)
heatmap.set_ylabel('Country', fontsize=12)
heatmap.set_title('Observations of Each Country by Year (2009 - 2013)', fontsize=14)

# Display the heatmap
plt.show()

# 6. Key Considerations for Selecting the Optimal Year Range

1. Completeness of Data:
    * 1970-2016: Only 2 countries have complete data. High missing values (656), which could lead to biased results and reduced accuracy.

    * 2005-2015: 11 years with fewer missing values (44). Only one country (Turkey) needs to be dropped to further reduce missing values to 33.

    * 2005-2014: 10 years with 38 missing values.

    * 2009-2013: 5 years with even fewer missing values (19). Dropping three countries (Turkey, Russia, New Zealand) reduces missing values to 4.
    
    * 2009-2012: 4 years with the least missing values (16). Dropping four countries (Turkey, Russia, New Zealand, United Kingdom) results in no missing values.

2. Impact on Analysis:
    *   More years generally provide a better trend analysis but come with the trade-off of more missing data.
    * Fewer years provide more complete data but may not capture long-term trends and changes effectively.

3. Number of Countries:
    * Dropping fewer countries is preferable to maintain a larger sample size for more robust comparative analysis.
    * However, if the number of dropped countries does not significantly impact the representation, it may be acceptable to drop them for a more complete dataset.

4. Business Case:
    * The focus on "Cost Optimization" requires reliable data to identify high and low spenders accurately.
    * Data completeness is critical to avoid biased conclusions and ensure accurate identification of best practices.


## Recommendations for Year Range Selection
Given the business case focused on "Cost Optimization," the goal is to identify reliable trends and best practices for pharmaceutical spending. The 2005-2014 range offers a balanced approach by providing a decade of data while reducing the impact of outliers like Russia's single observation in 2015.

### Conclusion:
RECOMMENDATION: Use the 2005-2014 range for analysis.
* The data is more complete, reducing bias and improving the accuracy of the analysis.
* A decade-long period is sufficient to capture meaningful trends and patterns.
* Excluding the outlier year (2015) helps in maintaining data integrity and reliability for cost optimization strategies.
* Drop 'NZL', 'TUR', 'RUS' and 'GBR'